In [24]:
import csv
import re
import pandas as pd
import time

import selenium 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.chrome.options import Options

In [25]:
sample_df = pd.read_csv("inxight_data_1.csv")
cas_df = sample_df[sample_df["CAS"] != "No CAS info"]
cas_df

,Unnamed: 0,CAS,URL,Highest Development Event Status and Year,Highest Development Event Source ID,Highest Development Event Source URL,Earliest Approved Event Status and Year,Earliest Approved Event Source ID,Earliest Approved Event Source URL,conditions,phases
0,0,623-84-7,https://drugs.ncats.io/drug/5Z492UNF9O,Possibly Marketed Outside US 1996,ANDA040166,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,Possibly Marketed Outside US 1996,ANDA040166,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,Bacillus cereus infection,Basic research
1,1,36653-82-4,https://drugs.ncats.io/drug/936JST6JCN,US Previously Marketed 1990,EXOSURF NEONATAL by GLAXOSMITHKLINE,https://www.accessdata.fda.gov/scripts/cder/da...,Possibly Marketed Outside US 1984,NU-DERM SUNFADER Skin Lightener with Sunscree...,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,"Dry, itchy skin; Respiratory distress syndrome...",Approved; Approved
2,2,112-92-5,https://drugs.ncats.io/drug/2KR89I4H1Y,US Previously Marketed,21 CFR 310.545(a)(18)(i)(B) skin protectant:w/...,https://www.gpo.gov/fdsys/pkg/CFR-2018-title21...,Possibly Marketed Outside US 1970,Ala Quin by Crown Laboratories,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,SUNBURN,Approved
3,3,328-50-7,https://drugs.ncats.io/drug/8ID597Z82X,Possibly Marketed Outside US,ALPHA-KETOGLUTARATE,http://naturaldatabase.therapeuticresearch.com...,NaN,NaN,NaN,Osteoporosis; Gastrointestinal microbiota imba...,Not Provided; Not Provided; Not Provided; Appr...
4,4,67-63-0,https://drugs.ncats.io/drug/ND2M416302,US Approved OTC,21 CFR 344.12 otic:ear drying aid isopropyl al...,https://www.gpo.gov/fdsys/pkg/CFR-2018-title21...,US Previously Marketed 1921,Rubbing Alcohol,https://books.google.com/books?id=1n4gAQAAMAAJ...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2316,2316,17141-74-1,https://drugs.ncats.io/drug/D652ZWF066,US Approved Rx 2018,NDA207078,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 2018,NDA207078,https://www.accessdata.fda.gov/scripts/cder/da...,NaN,NaN
2317,2317,118072-93-8,https://drugs.ncats.io/drug/70HZ18PH24,US Approved Rx 2013,ANDA091363,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 2001,ZOMETA by NOVARTIS,https://www.accessdata.fda.gov/scripts/cder/da...,Postmenopausal osteoporosis; Hypercalcemia; Pr...,Approved; Approved; Approved; Approved; Approved
2318,2318,139264-17-8,https://drugs.ncats.io/drug/2FS66TH3YW,US Approved Rx 2016,ANDA205074,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 1997,ZOMIG by IPR,https://www.accessdata.fda.gov/scripts/cder/da...,Migraine with aura; Migraine without aura,Approved; Approved
2319,2319,82626-48-0,https://drugs.ncats.io/drug/7K383OQI23,US Approved Rx 2016,ANDA201509,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 1992,NDA019908,https://www.accessdata.fda.gov/scripts/cder/da...,Insomnia,Approved


In [26]:
columns_to_check = [
    'URL',
    'Highest Development Event Status and Year',
    'Highest Development Event Source ID',
    'Highest Development Event Source URL',
    'Earliest Approved Event Status and Year',
    'Earliest Approved Event Source ID',
    'Earliest Approved Event Source URL',
    'conditions',
    'phases'
]

In [28]:
filtered_na = cas_df[cas_df[columns_to_check].isna().all(axis=1)]
cas_arr = filtered_na['CAS'].tolist()

In [29]:
def setup_webdriver():
    '''Initializes a headless selenium webdriver'''
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def url_inxight(cas):
    '''Generates an Inxight URL for a given CAS'''
    return f"https://drugs.ncats.io/substances?q={cas}"

# def url_inxight(cas):
#     '''Generates an Inxight URL for a given CAS'''
#     return f"https://drugs.ncats.io/substances?q=%22{cas}%22&facet=Substance%20Form%2FPrincipal%20Form"

In [30]:
def get_inxight_url(molecule, driver):
    '''
    Parameters
    ----------
    A molecule's CAS and an initialised webdriver.

    Returns
    -------
    The top Inxight search result for a given CAS number
    '''
    
    url = "N/A"  # Default in case of failure

    driver.get(url_inxight(molecule))
    try:
        WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'a[id="card-title"]')))
        elements = driver.find_elements(By.CSS_SELECTOR, 'a[id="card-title"]')
        if elements:
            element = elements[0]
            url = element.get_attribute('href')

    except (NoSuchElementException, TimeoutException):
        pass  

    return molecule, url

def get_best_urls(cas_arr):
    '''
    Parameters
    ----------
    An array of CAS values

    Returns
    -------
    An array of associated top matches
    '''
    
    best_matches = []
    driver = setup_webdriver()  
    try:
        for cas in cas_arr:
            molecule, url = get_inxight_url(cas, driver)
            best_matches.append((molecule, url))
    finally:
        driver.quit()  

    return best_matches

In [31]:
links = get_best_urls(cas_arr)

In [32]:
links

[('51022-69-6', 'https://drugs.ncats.io/drug/423W026MA9'),
 ('90-84-6', 'https://drugs.ncats.io/drug/Q94YYU22B8'),
 ('300-62-9', 'https://drugs.ncats.io/drug/CK833KGX7E'),
 ('20537-88-6', 'https://drugs.ncats.io/drug/ILA426L95O'),
 ('37517-28-5', 'https://drugs.ncats.io/drug/84319SGC3C'),
 ('2609-46-3', 'https://drugs.ncats.io/drug/7DZO8EB0Z3'),
 ('2097132-02-8', 'https://drugs.ncats.io/drug/71RR81V666'),
 ('90-45-9', 'https://drugs.ncats.io/drug/78OY3Z0P7Z'),
 ('60-32-2', 'https://drugs.ncats.io/drug/U6F3787206'),
 ('125-84-8', 'https://drugs.ncats.io/drug/0O54ZQ14I9'),
 ('106-60-5', 'https://drugs.ncats.io/drug/88755TAZ87'),
 ('317-34-0', 'https://drugs.ncats.io/drug/27Y3KJK423'),
 ('504-29-0', 'https://drugs.ncats.io/drug/WSX981HEWU'),
 ('65-49-6', 'https://drugs.ncats.io/drug/5B2658E0N2'),
 ('1951-25-3', 'https://drugs.ncats.io/drug/N3RQ532IUT'),
 ('50-48-6', 'https://drugs.ncats.io/drug/1806D8D52K'),
 ('68302-57-8', 'https://drugs.ncats.io/drug/BRL1C2459K'),
 ('88150-42-9', 'https

In [33]:
len(links)

833

In [34]:
identifiers = [url.split('/')[-1] for _, url in links]
identifiers = [identifier if identifier != 'A' else 'MISSING' for identifier in identifiers]
identifiers

['423W026MA9',
 'Q94YYU22B8',
 'CK833KGX7E',
 'ILA426L95O',
 '84319SGC3C',
 '7DZO8EB0Z3',
 '71RR81V666',
 '78OY3Z0P7Z',
 'U6F3787206',
 '0O54ZQ14I9',
 '88755TAZ87',
 '27Y3KJK423',
 'WSX981HEWU',
 '5B2658E0N2',
 'N3RQ532IUT',
 '1806D8D52K',
 'BRL1C2459K',
 '1J444QC288',
 '5138Q19F1X',
 '54S68520I4',
 'GWH6IJ239E',
 'R63VQ857OT',
 '9EM05410Q9',
 'CK833KGX7E',
 '7XU7A7DROE',
 '7C782967RD',
 '5S0W860XNR',
 'JUT23379TN',
 'H2HUX79FYK',
 '0F5N573A2Y',
 'K9X45X0051',
 '9013DUQ28K',
 '2Z07MYW1AZ',
 'BI009E452R',
 '20GCF755G6',
 'J0E2756Z7N',
 '9HLM53094I',
 'S747T1ERAJ',
 'H0G9379FGK',
 'DHA8014SS1',
 '80VUN7L00C',
 'O414PZ4LPZ',
 'O414PZ4LPZ',
 '97SEH7577T',
 '8GTS82S83M',
 '9IT35J3UV3',
 'T0LTO7L82X',
 '4T36H88UA7',
 '3Z9Y7UWC1J',
 'N21FAR7B4S',
 'O414PZ4LPZ',
 '843CEN85DI',
 'UP7QBP99PN',
 '1NF15YR6UY',
 'ba8b95ec',
 'SL4SX1O487',
 '80VUN7L00C',
 '19GW9E1WHN',
 'F91H02EBWT',
 'S866O45PIG',
 'BFE2NBZ7NX',
 '01YAE03M7J',
 'X32KK4201D',
 '3SCV180C9W',
 '9842X06Q6M',
 'O0ZR1R6RZ2',
 '004F72P8F4

In [35]:
# def extract_conditions_and_phases(data):
#     conditions_list = []
#     highest_approval_list = []

#     for entry in data:
#         if entry['name'] == 'Conditions' and 'value' in entry:
#             condition_info = entry['value']
            
#             # Extract the condition name
#             label = condition_info.get('label')
#             if label:
#                 conditions_list.append(label)
            
#             # Extract the highest phase of approval
#             highest_phase = condition_info.get('highestPhase')
#             if highest_phase:
#                 highest_approval_list.append(highest_phase)

#     return conditions_list, highest_approval_list

def extract_conditions_and_phases(data):
    """Extract conditions and their highest phases of approval from the API data."""
    conditions_list = []
    highest_approval_list = []

    for entry in data:
        if isinstance(entry, dict) and entry.get('name') == 'Conditions' and 'value' in entry:
            condition_info = entry['value']
            
            # Extract the condition name
            label = condition_info.get('label')
            if label:
                conditions_list.append(label)
            
            # Extract the highest phase of approval
            highest_phase = condition_info.get('highestPhase')
            if highest_phase:
                highest_approval_list.append(highest_phase)

    return conditions_list, highest_approval_list

In [36]:
def extract_event_details(data):
    event_details = {}
    
    for item in data:
        if 'value' in item and isinstance(item['value'], dict):  # Ensure 'value' is a dictionary
            details = item['value']
            if 'status' in details and 'sourceID' in details:
                if item['name'] == 'Highest Development Event' or item['name'] == 'Earliest Approved Event':
                    # Gather additional details
                    source_id = details.get('sourceID', 'No Source ID')
                    source_url = details.get('sourceURL', 'No Source URL')
                    
                    detail_info = {
                        'Status and Year': f"{details['status']} {details.get('year', '')}",
                        'Source ID': source_id,
                        'Source URL': source_url
                    }
                    
                    # Use the 'name' of the event as the key in the dictionary
                    event_details[item['name']] = detail_info
    
    return event_details

In [37]:
# def extract_events(identifiers):
#     events = []

#     for identifier in identifiers:
#         if identifier == "MISSING":
#             events.append({'Highest Development Event': {'Status and Year': 'Possibly Marketed Outside US 1996',
#    'Source ID': 'ANDA040166',
#    'Source URL': 'https://dailymed.nlm.nih.gov/dailymed/drugInfo.cfm?setid=c0ec47b7-c13a-4fa0-91fe-d7a03c70aec9'},
#   'Earliest Approved Event': {'Status and Year': 'Possibly Marketed Outside US 1996',
#    'Source ID': 'ANDA040166',
#    'Source URL': 'https://dailymed.nlm.nih.gov/dailymed/drugInfo.cfm?setid=c0ec47b7-c13a-4fa0-91fe-d7a03c70aec9'}})
#         data = get_additional_data(identifier)
#         print(extract_event_details(data))
#         events.append(extract_event_details(data))

#     return events
        
# events_arr = extract_events(identifiers)

def extract_events(identifiers):
    events = []
    acc = 0

    for identifier in identifiers:
        acc += 1
        print(acc)
        if identifier == "MISSING":
            # Create a dictionary where all details are 'MISSING'
            missing_details = {
                'Highest Development Event': {
                    'Status and Year': 'N/A',
                    'Source ID': 'N/A',
                    'Source URL': 'N/A'
                },
                'Earliest Approved Event': {
                    'Status and Year': 'N/A',
                    'Source ID': 'N/A',
                    'Source URL': 'N/A'
                }
            }
            events.append(missing_details)
        else:
            data = get_additional_data(identifier)
            event_details = extract_event_details(data)
            events.append(event_details)

    return events

In [38]:
def extract_conditions(identifiers):
    """Extract conditions for a list of identifiers."""
    conditions = []

    for identifier in identifiers:
        if identifier == "MISSING":
            conditions.append((['N/A'], ['N/A']))
        else:
            data = get_additional_data(identifier)
            if data:  # Check if data is not empty
                extracted_data = extract_conditions_and_phases(data)
                conditions.append(extracted_data)
            else:
                conditions.append(([], []))  # Append empty lists if no data
    return conditions

In [39]:
import requests

def get_additional_data(id):
    r = requests.get(f'https://drugs.ncats.io/api/v1/substances({id})/@additional')
    if 200 == r.status_code:
        return r.json()
    return '{}'

In [40]:
events_arr = extract_events(identifiers)
events_arr

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


[{'Highest Development Event': {'Status and Year': 'US Approved Rx 2003',
   'Source ID': 'ANDA076065',
   'Source URL': 'https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo=076065'},
  'Earliest Approved Event': {'Status and Year': 'US Previously Marketed 1979',
   'Source ID': 'CYCLOCORT by ASTELLAS',
   'Source URL': 'https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo=018116'}},
 {'Highest Development Event': {'Status and Year': 'US Approved Rx 2011',
   'Source ID': 'ANDA091680',
   'Source URL': 'https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo=091680'},
  'Earliest Approved Event': {'Status and Year': 'US Previously Marketed 1959',
   'Source ID': 'TEPANIL by 3M',
   'Source URL': 'https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo=011673'}},
 {'Highest Development Event': {'Status and Year': 'US Approved Rx 2003',
   'Source ID': 'A

In [41]:
import requests

def get_additional_data(id):
    r = requests.get(f'https://drugs.ncats.io/api/v1/substances({id})/@additional')
    if 200 == r.status_code:
        return r.json()
    return ([], [])

In [42]:
conditions_arr = extract_conditions(identifiers)
conditions_arr

[(['Corticosteroid-responsive dermatoses'], ['Approved']),
 (['Unknown', 'Exogenous obesity', 'Unknown'], ['Approved']),
 (['Attention deficit hyperactivity disorder'], ['Approved']),
 (['Ovarian cancer', 'Head and neck cancer'], ['Approved', 'Approved']),
 (['Cystic fibrosis',
   'Sepsis',
   'Urinary tract infections',
   'Meningitis',
   'Pneumonia'],
  ['Phase III', 'Approved', 'Approved', 'Approved', 'Phase III']),
 (['Hypertension', 'Heart failure'], ['Approved', 'Approved']),
 ([], []),
 (['Adult T-cell leukemia/lymphoma', 'Vaginal trichomoniasis', 'Mycoses'],
  ['Basic research', 'Phase I', 'Approved']),
 (['Bleeding'], ['Approved']),
 (['Cushing’s disease'], ['Approved']),
 (['Condylomata acuminata', 'Malignant glioma', 'Actinic keratosis'],
  ['Approved', 'Approved', 'Approved']),
 (['Airway obstruction'], ['Approved']),
 ([], []),
 (['Tuberculosis', "Crohn's disease", 'Ulcerative colitis'],
  ['Approved', 'Phase II', 'Phase II']),
 (['Ventricular tachycardia', 'Recurring ven

In [43]:
data = []
for event in events_arr:
    record = {}
    for key, value in event.items():
        for sub_key, sub_value in value.items():
            record[f"{key} {sub_key}"] = sub_value
    data.append(record)

df = pd.DataFrame(data)

In [44]:
rows = [{'conditions': conditions, 'phases': phases} for conditions, phases in conditions_arr]

# Create DataFrame
conditions_df = pd.DataFrame(rows)
conditions_df['conditions'] = conditions_df['conditions'].apply(lambda x: '; '.join(x))
conditions_df['phases'] = conditions_df['phases'].apply(lambda x: '; '.join(x))
conditions_df

,conditions,phases
0,Corticosteroid-responsive dermatoses,Approved
1,Unknown; Exogenous obesity; Unknown,Approved
2,Attention deficit hyperactivity disorder,Approved
3,Ovarian cancer; Head and neck cancer,Approved; Approved
4,Cystic fibrosis; Sepsis; Urinary tract infecti...,Phase III; Approved; Approved; Approved; Phase...
...,...,...
828,Insomnia,Approved
829,Influenza virus infections,Approved
830,Vision disorders; Cataract; Age-related memory...,Approved; Phase III; Phase III
831,severe chronic pain,Approved


In [45]:
merged = pd.concat([df, conditions_df], axis=1)
merged

,Highest Development Event Status and Year,Highest Development Event Source ID,Highest Development Event Source URL,Earliest Approved Event Status and Year,Earliest Approved Event Source ID,Earliest Approved Event Source URL,conditions,phases
0,US Approved Rx 2003,ANDA076065,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 1979,CYCLOCORT by ASTELLAS,https://www.accessdata.fda.gov/scripts/cder/da...,Corticosteroid-responsive dermatoses,Approved
1,US Approved Rx 2011,ANDA091680,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 1959,TEPANIL by 3M,https://www.accessdata.fda.gov/scripts/cder/da...,Unknown; Exogenous obesity; Unknown,Approved
2,US Approved Rx 2003,ANDA040480,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 1931,Benzedrine Inhaler,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,Attention deficit hyperactivity disorder,Approved
3,US Approved Rx 1995,NDA020221,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 1995,NDA020221,https://www.accessdata.fda.gov/scripts/cder/da...,Ovarian cancer; Head and neck cancer,Approved; Approved
4,US Approved Rx 1993,ANDA064045,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 1976,AMIKIN by APOTHECON,https://www.accessdata.fda.gov/scripts/cder/da...,Cystic fibrosis; Sepsis; Urinary tract infecti...,Phase III; Approved; Approved; Approved; Phase...
...,...,...,...,...,...,...,...,...
828,US Approved Rx 2008,ANDA077505,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 1999,NDA020859,https://www.accessdata.fda.gov/scripts/cder/da...,Insomnia,Approved
829,US Approved Rx 1999,NDA021036,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 1999,NDA021036,https://www.accessdata.fda.gov/scripts/cder/da...,Influenza virus infections,Approved
830,Possibly Marketed Outside US,"Ocuvel by Adler-Stern Pharmaceuticals, LLC",https://www.drugbank.ca/drugs/DB11176,Possibly Marketed Outside US 2014,"Ocuvel by Adler-Stern Pharmaceuticals, LLC",https://dailymed.nlm.nih.gov/dailymed/drugInfo...,Vision disorders; Cataract; Age-related memory...,Approved; Phase III; Phase III
831,US Approved Rx 2004,NDA021060,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 2004,NDA021060,https://www.accessdata.fda.gov/scripts/cder/da...,severe chronic pain,Approved


In [46]:
links_df = pd.DataFrame(links, columns=['CAS', 'URL'])
links_df

,CAS,URL
0,51022-69-6,https://drugs.ncats.io/drug/423W026MA9
1,90-84-6,https://drugs.ncats.io/drug/Q94YYU22B8
2,300-62-9,https://drugs.ncats.io/drug/CK833KGX7E
3,20537-88-6,https://drugs.ncats.io/drug/ILA426L95O
4,37517-28-5,https://drugs.ncats.io/drug/84319SGC3C
...,...,...
828,151319-34-5,https://drugs.ncats.io/drug/S62U433RMH
829,139110-80-8,https://drugs.ncats.io/drug/L6O3XI777I
830,144-68-3,https://drugs.ncats.io/drug/CV0IB81ORO
831,107452-89-1,https://drugs.ncats.io/drug/7I64C51O16


In [47]:
result = pd.concat([links_df, merged], axis=1)
result

,CAS,URL,Highest Development Event Status and Year,Highest Development Event Source ID,Highest Development Event Source URL,Earliest Approved Event Status and Year,Earliest Approved Event Source ID,Earliest Approved Event Source URL,conditions,phases
0,51022-69-6,https://drugs.ncats.io/drug/423W026MA9,US Approved Rx 2003,ANDA076065,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 1979,CYCLOCORT by ASTELLAS,https://www.accessdata.fda.gov/scripts/cder/da...,Corticosteroid-responsive dermatoses,Approved
1,90-84-6,https://drugs.ncats.io/drug/Q94YYU22B8,US Approved Rx 2011,ANDA091680,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 1959,TEPANIL by 3M,https://www.accessdata.fda.gov/scripts/cder/da...,Unknown; Exogenous obesity; Unknown,Approved
2,300-62-9,https://drugs.ncats.io/drug/CK833KGX7E,US Approved Rx 2003,ANDA040480,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 1931,Benzedrine Inhaler,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,Attention deficit hyperactivity disorder,Approved
3,20537-88-6,https://drugs.ncats.io/drug/ILA426L95O,US Approved Rx 1995,NDA020221,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 1995,NDA020221,https://www.accessdata.fda.gov/scripts/cder/da...,Ovarian cancer; Head and neck cancer,Approved; Approved
4,37517-28-5,https://drugs.ncats.io/drug/84319SGC3C,US Approved Rx 1993,ANDA064045,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 1976,AMIKIN by APOTHECON,https://www.accessdata.fda.gov/scripts/cder/da...,Cystic fibrosis; Sepsis; Urinary tract infecti...,Phase III; Approved; Approved; Approved; Phase...
...,...,...,...,...,...,...,...,...,...,...
828,151319-34-5,https://drugs.ncats.io/drug/S62U433RMH,US Approved Rx 2008,ANDA077505,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 1999,NDA020859,https://www.accessdata.fda.gov/scripts/cder/da...,Insomnia,Approved
829,139110-80-8,https://drugs.ncats.io/drug/L6O3XI777I,US Approved Rx 1999,NDA021036,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 1999,NDA021036,https://www.accessdata.fda.gov/scripts/cder/da...,Influenza virus infections,Approved
830,144-68-3,https://drugs.ncats.io/drug/CV0IB81ORO,Possibly Marketed Outside US,"Ocuvel by Adler-Stern Pharmaceuticals, LLC",https://www.drugbank.ca/drugs/DB11176,Possibly Marketed Outside US 2014,"Ocuvel by Adler-Stern Pharmaceuticals, LLC",https://dailymed.nlm.nih.gov/dailymed/drugInfo...,Vision disorders; Cataract; Age-related memory...,Approved; Phase III; Phase III
831,107452-89-1,https://drugs.ncats.io/drug/7I64C51O16,US Approved Rx 2004,NDA021060,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 2004,NDA021060,https://www.accessdata.fda.gov/scripts/cder/da...,severe chronic pain,Approved


In [48]:
result.tail(10)

,CAS,URL,Highest Development Event Status and Year,Highest Development Event Source ID,Highest Development Event Source URL,Earliest Approved Event Status and Year,Earliest Approved Event Source ID,Earliest Approved Event Source URL,conditions,phases
823,526-36-3,https://drugs.ncats.io/drug/WPY40FTH8K,US Approved OTC,21 CFR 341.20(b)(10) cough/cold:nasal deconges...,https://www.gpo.gov/fdsys/pkg/CFR-2018-title21...,US Previously Marketed 1959,Otrivin by Ciba Geigy,OB NME Appendix 1950-1985,Rhinitis; Allergic rhinitis; Common cold,Approved; Approved; Approved
824,10257-31-5,https://drugs.ncats.io/drug/2OYE00PC25,US Approved Rx 2023,NDA215830,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 2023,NDA215830,https://www.accessdata.fda.gov/scripts/cder/da...,,
825,146-48-5,https://drugs.ncats.io/drug/2Y49VWD90Q,US Previously Marketed,21 CFR 310.528(a) aphrodisiac yohimbinum,https://www.gpo.gov/fdsys/pkg/CFR-2018-title21...,Possibly Marketed Outside US 2015,NADA140866,https://dailymed.nlm.nih.gov/dailymed/drugInfo...,Erectile dysfunction,Approved
826,107753-78-6,https://drugs.ncats.io/drug/XZ629S5L50,NaN,NaN,NaN,NaN,NaN,NaN,Asthma,Approved
827,7481-89-2,https://drugs.ncats.io/drug/6L3XT8CB3I,US Previously Marketed 1992,HIVID by ROCHE,https://www.accessdata.fda.gov/scripts/cder/da...,US Previously Marketed 1992,HIVID by ROCHE,https://www.accessdata.fda.gov/scripts/cder/da...,HIV infection,Approved
828,151319-34-5,https://drugs.ncats.io/drug/S62U433RMH,US Approved Rx 2008,ANDA077505,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 1999,NDA020859,https://www.accessdata.fda.gov/scripts/cder/da...,Insomnia,Approved
829,139110-80-8,https://drugs.ncats.io/drug/L6O3XI777I,US Approved Rx 1999,NDA021036,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 1999,NDA021036,https://www.accessdata.fda.gov/scripts/cder/da...,Influenza virus infections,Approved
830,144-68-3,https://drugs.ncats.io/drug/CV0IB81ORO,Possibly Marketed Outside US,"Ocuvel by Adler-Stern Pharmaceuticals, LLC",https://www.drugbank.ca/drugs/DB11176,Possibly Marketed Outside US 2014,"Ocuvel by Adler-Stern Pharmaceuticals, LLC",https://dailymed.nlm.nih.gov/dailymed/drugInfo...,Vision disorders; Cataract; Age-related memory...,Approved; Phase III; Phase III
831,107452-89-1,https://drugs.ncats.io/drug/7I64C51O16,US Approved Rx 2004,NDA021060,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 2004,NDA021060,https://www.accessdata.fda.gov/scripts/cder/da...,severe chronic pain,Approved
832,30516-87-1,https://drugs.ncats.io/drug/4B9XT59T7S,US Approved Rx 2008,ANDA077981,https://www.accessdata.fda.gov/scripts/cder/da...,US Approved Rx 1987,NDA019655,https://www.accessdata.fda.gov/scripts/cder/da...,HIV-1 infection,Approved


In [49]:
result.to_csv("inxight_data_2.csv")

In [ ]:
result.to_csv("inxight_data_2.csv")